In [9]:
import config
import numpy as np
import pandas as pd
import tensorflow as tf
import pickle
import config
from tensorflow import keras
import os
from sklearn.preprocessing import StandardScaler
from tensorflow.keras import initializers
from tensorflow.keras import optimizers
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Conv2D,AveragePooling2D, BatchNormalization,Dropout,Flatten,Dense
from tensorflow.keras.models import load_model
from tensorflow.keras.models import Model
import matplotlib.pyplot as plt
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy.util as util

from sklearn.metrics import pairwise_distances_argmin_min, pairwise_distances
plt.style.use('ggplot')


%config InlineBackend.figure_format = 'retina'
%matplotlib inline
import io
import librosa
import librosa.display
import soundfile as sf
import glob


import pydub
from urllib.request import urlopen
import requests
from shutil import copyfileobj
from tempfile import NamedTemporaryFile
from urllib.request import urlopen, Request
from implicit.als import AlternatingLeastSquares
from scipy.sparse import coo_matrix

In [2]:
music_df =pd.read_csv('music_values.csv')

# Latent Features 

These functions are to enable us to find similar songs even if they do not appear in the mixesdb dataset. This will happen quite a lot as there are many songs out there.


im in two minds about whether I should scale the data or not. Im sort of thinking if the neural net gave bigger numbers to certain features it may mean that they should have bigger influence anyway. I will try both ways to see which one produces better results, although very subjective what best results would look like.


Also I was wondering if I should use Cosine similarity vs Euclidean distance so did a bit of reading and found this article -https://cmry.github.io/notes/euclidean-v-cosine . Bascially says that if we care about magnitude we should use Euclidean, and I think we do care about magnitude.

In [3]:
client_id= config.client_id
client_secret =config.client_secret

username = '1143043561'
auth = SpotifyClientCredentials(
    client_id=client_id,
    client_secret=client_secret
)


try:
    token = auth.get_access_token()
except:
    os.remove(f'.cache-{username}')
    token = auth.get_access_token()
    
#create spotify object
spotify = spotipy.Spotify(auth=token)

In [4]:
def closest_k_nodes(node, nodes,k=5):
    node=node.reshape(1,-1)
    dist_2 = pairwise_distances(nodes,node)
    smallest_inds=np.argsort(dist_2[:,0])[:k]
    smallest_dists=dist_2[smallest_inds]
    return (smallest_inds, smallest_dists)

In [5]:
def find_similar(song_id,song_list=music_df,num_songs=5,diff_genre='no',same_key='yes',similar_tempo='yes',margin=3,scaled=False):
    
    
    song_index=song_list[song_list.id==song_id].index[0]
    
   
    orig_key = song_list.loc[song_index,'key']
    orig_tempo= song_list.loc[song_index, 'tempo']
    orig_genre= song_list.loc[song_index,'genre_approx']
    node = song_list.loc[song_index]
    
    #check if you want songs of same key
    if same_key=='yes':
    #if yes then filter out other keys    
        print(f'key:{orig_key}')
        song_list = song_list[song_list.key ==orig_key]
    #can also enter number to specify what key you want    
    elif type(same_key) !=str:
        song_list = song_list[song_list.key==same_key]


    # check if you want similar tempo    
    if similar_tempo=='yes':
        print(f'tempo:{orig_tempo}')
        #if yes can also specify how similar you want it 
        song_list=song_list[song_list.tempo.between(int(orig_tempo)-margin,int(orig_tempo)+margin)]
        
    elif type(similar_tempo) !=str:
        #can also specify a specific tempo that you want
        song_list = song_list[song_list.tempo.between(int(similar_tempo)-margin,int(similar_tempo)+margin)]
    
    #check if you want a different genre
    if diff_genre=='yes':
        print(f'genre:{orig_genre}')
        song_list= song_list[song_list.genre_approx != orig_genre]
        
    
    #adds requested song back into dataframe in case it was filtered out
    song_list=song_list.append(node,ignore_index=True)
    song_id_info=song_list.copy()
    #makes sure certain columns are ints as they come as strings from spotify
    song_list.key=song_list.key.astype(int)
    song_list['mode']=song_list['mode'].astype(int)
    song_list.time_signature=song_list.time_signature.astype(int)
    #exclude any other column which is a string
    song_list=song_list.select_dtypes(exclude=object)
    
    #scale the data
    if scaled==True:
        scaler = StandardScaler()
        song_list = scaler.fit_transform(song_list)

        node = song_list[-1]
        nodes = song_list
    
    else:
        node=np.asarray(song_list.iloc[-1])
        nodes=song_list
    
    #find closest song using euclidean distance
    closest_inds,distances = closest_k_nodes(node,nodes,num_songs)
    
    #get song ids of closest songs
    song_ids=song_id_info.loc[closest_inds,'id']
    #make sure there is no NANs 
    song_ids=[x for x in song_ids if str(x) != 'nan']
    
    #find track names using spotify API
    tracks=spotify.tracks(list(song_ids))
    song_names=[]
    for track in tracks['tracks']:
        song_names.append((track['name'],track['artists'][0]['name']))

    #create dataframe of songs and their euclidean distance from the original song
    close_song_df=pd.DataFrame({'song_ids':song_ids,
                                'song_names':song_names,
                                'distances':distances.flatten()})
    return (close_song_df)

NameError: name 'music_df' is not defined

In [ ]:
def spectrogram_then_latent(url, song_id, name, model):
    '''basically the same as the function i used to
    create a load of spectrograms but just for 1 this time
    takes  url and turns the preview mp3 
        into a spectrogram and then uses a model to 
        extract latent features'''   

    if url != None:
        try:
            mp3_url = url
            wav = io.BytesIO()
            with urlopen(mp3_url) as r:
                r.seek = lambda *args: None  # allow pydub to call seek(0)
                pydub.AudioSegment.from_file(r).export(wav, "wav")

            wav.seek(0)
            y, sr = librosa.load(wav)

            # mel-scaled power (energy-squared) spectrogram
            mel_spec = librosa.feature.melspectrogram(y,
                                                      sr=sr,
                                                      n_mels=128,
                                                      hop_length=1024,
                                                      n_fft=2048)
            # Convert to log scale (dB). We'll use the peak power as reference.
            log_mel_spec = librosa.amplitude_to_db(mel_spec, ref=np.max)
            #make dimensions of the array smaller
            log_mel_spec = np.resize(log_mel_spec, (128, 644))

            log_mel_spec_arr = log_mel_spec.reshape(
                log_mel_spec.shape[0], log_mel_spec.shape[1], 1)
            pre_process = np.expand_dims(log_mel_spec_arr, axis=0)
            pre_process = pre_process / 255
            latent = model.predict(pre_process)
            

        except:
            print('song has no useable url')
            return
    
        #just going to keep it as a df for ease of use even though only 1 row
        latent_df= pd.DataFrame(latent)
        latent_df=latent_df.loc[~(latent_df==0).all(axis=1)]
        latent_df['song_names']= name
        latent_df['id']= song_id
    
    else:
        print('song has no useable url')
        return
    
    return (latent_df)

In [ ]:
def get_music_quals(id_list):
    '''get music qualities such as tempo and time sig
        with a list of song ids'''
    qualities={}
    count=0
    for song in id_list:
        if type(song)==str:
            quals=spotify.audio_features(song)
            try:
                qualities[song]=quals
            except:
                print(count)
                count+=1
    return qualities

In [6]:
def get_spotify_quals(music_df):
    '''get spotify music qualities and put into
        dataframe then merge into main dataframe'''
    #get ids of songs so can search spotify
    id_list=list(music_df['id'].values)
    #earlier function that helps look for music quals
    quals = get_music_quals(id_list)
    #put the qualities as column headers
    df=pd.DataFrame(columns=list(quals[list(quals.keys())[0]][0].keys()))
    for ind, key in enumerate(quals.keys()):
    #iterate over songs and get the spotify qualities for each song into the df
        song=quals[key]
        try:
            df.loc[ind]=list(song[0].values())
        except:
            print(song)
    #merge the two dataframes together on id and return
    orig_and_spotify =pd.merge(music_df,df,on='id',how='outer')
    
    #drop columns we dont really need
    orig_and_spotify.drop(columns=['track_href','analysis_url','duration_ms','type','uri'],inplace=True)
    #turning some of the columns into ints so can be used
    #using try and except as there may be some NANs
    try:
        orig_and_spotify.key=orig_and_spotify.key.astype(int)
        orig_and_spotify['mode']=orig_and_spotify['mode'].astype(int)
        orig_and_spotify.time_signature=orig_and_spotify.time_signature.astype(int)
        return orig_and_spotify
    except:
        return orig_and_spotify
    

Wanted to put the whole process into one function so can be used really easily and to be able to handle new songs really easily without the really long process.

# Collaborative Model

This section is for when a song is found within our mixes dataset - here we are going to use alternating least squares. The biggest reason I chose this form of matrix factorization over something like logistic matrix factorization is that I want to eventually re do this project in Spark for practice and Spark also has an alternating least squares library.

In [10]:
collab_df=pd.read_csv('../song_collab_filtering/mixesdb_df_for_recs.csv')
collab_df.drop(columns=['Unnamed: 0'],inplace=True)
plays= collab_df['size']
users=collab_df.user_nums
songs=collab_df.song_nums
Matrix_us=coo_matrix((plays, (songs, users))).tocsr()

In [11]:
#needed to make the training not take ages
os.environ["MKL_NUM_THREADS"] = "1"

In [12]:
collab_model = AlternatingLeastSquares(factors=30)

In [13]:
collab_model.fit(Matrix_us)

In [19]:
a=collab_model.similar_items(1)

In [20]:
a=[val[0] for val in a]

In [30]:
collab_df.columns

Index(['set_title', 'set_list', 'artist', 'song', 'spotify_song_name',
       'spotify_id', 'preview', 'songs', 'set_title_split', 'user_nums',
       'song_nums', 'size'],
      dtype='object')

In [ ]:
def collab_filter(song_id, user_song_df, num_songs=5):
    '''
    song_id = spotify id for individual song
    user_song_df= dataframe with users, songs, playcounts etc
    for the time being i am not going to enable filtering by key/tempo as not enough songs
    but in future will do
    '''

    song_num = song_list[song_list.spotify_id == song_id].song_nums.values[0]
    print(song_num)
    print(type(song_num))
    #orig_key = song_list[song_list.spotify_id==song_id].key.values[0]
    #orig_tempo= song_list[song_list.spotify_id==song_id].tempo.values[0]

    #check if you want songs of same key
    #if same_key=='yes':
    #if yes then filter out other keys
    #    print(f'key:{orig_key}')
    #    song_list = song_list[song_list.key ==orig_key]

    #can also enter number to specify what key you want
    # elif type(same_key) !=str:
    #     song_list = song_list[song_list.key==same_key]

    # check if you want similar tempo
    #  if similar_tempo=='yes':
    #     print(f'tempo:{orig_tempo}')
    #if yes can also specify how similar you want it
    #     lower= int(orig_tempo)-margin
    #    higher=int(orig_tempo)+margin
    #    song_list=song_list[song_list.tempo.between(lower,higher)]

    #elif type(similar_tempo) !=str:
    #can also specify a specific tempo that you want
    #   song_list = song_list[song_list.tempo.between(int(similar_tempo)-margin,int(similar_tempo)+margin)]

    # refined_ids=song_list.spotify_id

    user_song_refined = user_song_df[user_song_df.spotify_id.isin(
        refined_ids)].copy()

    plays = user_song_refined['size']
    user_nums = user_song_refined.user_nums
    song_nums = user_song_refined.song_nums

    B = coo_matrix((plays, (song_nums, user_nums))).tocsr()

    model = AlternatingLeastSquares(factors=30)
    model.fit(B)
    songs_inds = model.similar_items(song_num, N=num_songs)
    songs_inds = [tup[0] for tup in songs_inds]

    return song_list[song_list.song_nums.isin(songs_inds)]

# Full Process

In [9]:
#load pretrained model
model = load_model('more_data_08-2.07.h5')
# take the last layer off the model so we can get to the latent features
new_model =model
new_model.layers.pop()
new_model_2 = Model(new_model.input, new_model.layers[-3].output)

In [10]:
music_df=pd.read_csv('music_values.csv')

In [11]:
def get_similar_for_new(song_id,
                        collab_df,
                        music_df,
                        model,
                        num_songs=5,
                        diff_genre='no',
                        same_key='yes',
                        similar_tempo='yes',
                        margin=3,
                        scaled=False):
    '''
        First checks if song in collaborative filtering database
        if not it then checks if song id in music dataframe and 
        if song is in neither tries to create the spectrogram 
        and find spotify features for the specified song, 
        then finds similar songs to that song using the 
        aforementioned qualities
    '''

    if song_id in collab_df.spotify_id.unique():
        return collab_filter
    
    #checks if song in music dataframe
    if song_id in music_df.id.unique():
        #if it is in the dataframe can just find similar songs
        close_song_df = find_similar(song_id,
                                     music_df,
                                     num_songs=num_songs,
                                     diff_genre=diff_genre,
                                     same_key=same_key,
                                     similar_tempo=similar_tempo,
                                     margin=margin,
                                     scaled=scaled)

        return close_song_df

    else:
        track = spotify.track(song_id)
        song_name = track['name']
        preview_url = track['preview_url']
        #check if the song has a url
        if preview_url:
            single_song_df = spectrogram_then_latent(preview_url, song_id,
                                                     song_name, model)

            #can just use the same spotify function as before
            single_song_with_spotify = get_spotify_quals(single_song_df)

            #sometimes the names of the columns can become strings/ints so need to check their the same
            cols = [str(col) for col in list(single_song_with_spotify.columns)]

            single_song_with_spotify.columns = cols
            #need to remove genre approx in the future
            single_song_with_spotify['genre_approx'] = 10
            single_song_with_spotify = single_song_with_spotify.reindex(sorted(
                single_song_with_spotify.columns),
                                                                        axis=1)
            music_df = pd.concat([music_df, single_song_with_spotify],
                                 ignore_index=True)
            #now we have song in dataframe can just do same process as the first bit of if statement
            close_song_df = find_similar(song_id,
                                         music_df,
                                         num_songs=num_songs,
                                         diff_genre=diff_genre,
                                         same_key=same_key,
                                         similar_tempo=similar_tempo,
                                         margin=margin,
                                         scaled=scaled)
            return close_song_df

        else:
            print('song has no url/ no spotify info')
            return

In [35]:
get_similar_for_new('7KA4W4McWYRpgf0fWsJZWB',music_df,new_model_2,same_key='yes',num_songs=10)

key:6
tempo:78.558


,song_ids,song_names,distances
0,7KA4W4McWYRpgf0fWsJZWB,"(See You Again (feat. Kali Uchis), Tyler, The ...",0.000000
1,7KA4W4McWYRpgf0fWsJZWB,"(See You Again (feat. Kali Uchis), Tyler, The ...",0.000000
2,2RwS1tNuq1eKJvhxaZKp3x,"(Olles leiwand, TTR Allstars)",7.220106
3,7ymbpIW2rzSKxRrtpWN80a,"(Oh She, Samuel Gregory)",9.698834
4,0owUIdKletsUs6kMUlr32L,"(reach out, Sweeps)",10.943082
5,2ysQz5RXNkX8ylV0WjbFDj,"(Cypherspace, Da Staummtisch feat. Linz Rap Al...",11.383145
6,3iJAySzjUixQC1oWBzmPji,"(october, oofoe)",11.454706
7,68uaEf2jgIsUKwM9RrOiK6,"(Good Thing, Jeangu Macrooy)",12.320747
8,4GxU9RLeOS2qqoj1Em53Ly,"(2 Far, Dizzee Rascal)",12.550152
9,38igNG6Dz0dJZU7AliitGJ,"(Hugging You (Quietly), Jazzinuf)",12.649486


In [33]:
find_similar('1BA4OdphRNKQSeHrMHLAWg',music_df,diff_genre='no',same_key='yes',num_songs=10)


IndexError: index 0 is out of bounds for axis 0 with size 0

In [ ]:
'4KjNEsQ4jkqXwCmsn71a9p'